In [42]:
import os
import numpy as np
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import LSTM, Embedding,Dense


In [4]:
file1 = open("./input.txt","r")
songs=file1.read()


In [5]:
songs

'X: 1\nT: The Enchanted Valley\nM: 2/4\nL: 1/16\nB: "O\'Neill\'s 1"\nN: "Very slow" "collected by J. O\'Neill"\nN:\nZ: "Transcribed by Norbert Paap, norbertp@bdu.uva.nl"\nZ:\nK:Gm\nG3-A (Bcd=e) | f4 (g2dB) | ({d}c3-B) G2-E2 | F4 (D2=E^F) |\nG3-A (Bcd=e) | f4 d2-f2 | (g2a2 b2).g2 | {b}(a2g2 f2).d2 |\n(d2{ed}c2) B2B2 | (A2G2 {AG}F2).D2 | (GABc) (d2{ed}c>A) | G2G2 G2z ||\nG | B2c2 (dcAB) | G2G2 G3G | B2d2 (gfdc) | d2g2 (g3ga) |\n(bagf) (gd)d>c | (B2AG) F-D.D2 | (GABc) d2d2 | (bgfd) cA.F2 |\nG2A2 (B2{cB}AG) | A3-G F2-D2 | (GABc) (d2{ed}c>A) | G2G2 G2z2 ||\n\n\nX: 2\nT: Fare You Well\nM: 2/4\nL: 1/16\nB: "O\'Neill\'s 2"\nN: "Slow" "collected by F. O\'Neill"\nZ: "Transcribed by Norbert Paap, norbertp@bdu.uva.nl"\nK:D\nf-g | a3-b g3-a | f4 e3-d | d3-c A3-B | c4 d3-e |\nd3-c (3(A2G2F2) | G4F2-G2 | A-d3 d3-e | d6 ||\nA2 | d3-e f3-g | a4 a3-g | a3-b a3-f | g4 g3-g |\na3-b a3-g | {e}=f4 e3-c | d3-c A3-G | A6 f-g |\na3-b g3-a | f4 e3-d | d3-c A3-B | c4 d3-e |\nd3-c (3(A2G2F2) | G4 F2-G2 | A-d3 d3-

In [9]:
vocab = sorted(set(songs))
len(vocab)

95

In [12]:
### Define numerical representation of text ###

# Create a mapping from character to unique index.
# For example, to get the index of the character "d", 
#   we can evaluate `char2idx["d"]`.  
char2idx = {u:i for i, u in enumerate(vocab)}

# Create a mapping from indices to characters. This is
#   the inverse of char2idx and allows us to convert back
#   from unique index to the character in our vocabulary.
idx2char = np.array(vocab)

In [14]:
idx2char

array(['\t', '\n', ' ', '!', '"', '#', '%', '&', "'", '(', ')', '*', '+',
       ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
       'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
       'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
       '{', '|', '}', '~'], dtype='<U1')

In [15]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  '+' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  ...
}


In [16]:
### Vectorize the songs string ###

'''TODO: Write a function to convert the all songs string to a vectorized
    (i.e., numeric) representation. Use the appropriate mapping
    above to convert from vocab characters to the corresponding indices.

  NOTE: the output of the `vectorize_string` function 
  should be a np.array with `N` elements, where `N` is
  the number of characters in the input string
'''
def vectorize_string(string):
  vectorized_output = np.array([char2idx[char] for char in string])
  return vectorized_output

# def vectorize_string(string):
  # TODO

vectorized_songs = vectorize_string(songs)

In [19]:
vectorized_songs

array([57, 27,  2, ..., 27, 92,  1])

In [21]:
print ('{} ---- characters mapped to int ----> {}'.format(repr(songs[:10]), vectorized_songs[:10]))
# check that vectorized_songs is a numpy array
assert isinstance(vectorized_songs, np.ndarray), "returned result should be a numpy array"

'X: 1\nT: Th' ---- characters mapped to int ----> [57 27  2 18  1 53 27  2 53 72]


In [27]:
### Batch definition to create training examples ###

def get_batch(vectorized_songs, seq_length, batch_size):
  # the length of the vectorized songs string
  n = vectorized_songs.shape[0] - 1
  # randomly choose the starting indices for the examples in the training batch
  idx = np.random.choice(n-seq_length, batch_size)

  '''TODO: construct a list of input sequences for the training batch'''
  input_batch = [vectorized_songs[i : i+seq_length] for i in idx]
  # input_batch = # TODO
  '''TODO: construct a list of output sequences for the training batch'''
  output_batch = [vectorized_songs[i+1 : i+seq_length+1] for i in idx]
  # output_batch = # TODO

  # x_batch, y_batch provide the true inputs and targets for network training
  x_batch = np.reshape(input_batch, [batch_size, seq_length])
  y_batch = np.reshape(output_batch, [batch_size, seq_length])
  return x_batch, y_batch


# Perform some simple tests to make sure your batch function is working properly! 
test_args = (vectorized_songs, 10, 2)


In [26]:
test_args

(array([57, 27,  2, ..., 27, 92,  1]), 10, 2)

In [28]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)

for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step   0
  input: 83 ('s')
  expected output: 2 (' ')
Step   1
  input: 2 (' ')
  expected output: 29 ('<')
Step   2
  input: 29 ('<')
  expected output: 74 ('j')
Step   3
  input: 74 ('j')
  expected output: 67 ('c')
Step   4
  input: 67 ('c')
  expected output: 33 ('@')


In [34]:
def build_LSTM(rnn_units): 
  return LSTM(
    rnn_units, 
    return_sequences=True, 
    recurrent_initializer='glorot_uniform',
    recurrent_activation='sigmoid',
    stateful=True,
  )

In [35]:
### Defining the RNN Model ###

'''TODO: Add LSTM and Dense layers to define the RNN model using the Sequential API.'''
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential([
    # Layer 1: Embedding layer to transform indices into dense vectors 
    #   of a fixed embedding size
    Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),

    # Layer 2: LSTM with `rnn_units` number of units. 
    # TODO: Call the LSTM function defined above to add this layer.
    build_LSTM(rnn_units), 
    # LSTM('''TODO'''),

    # Layer 3: Dense (fully-connected) layer that transforms the LSTM output
    #   into the vocabulary size. 
    # TODO: Add the Dense layer.
    Dense(vocab_size)
    # '''TODO: DENSE LAYER HERE'''
  ])

  return model

# Build a simple model with default hyperparameters. You will get the 
#   chance to change these later.
model = build_model(len(vocab), embedding_dim=256, rnn_units=1024, batch_size=32)

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (32, None, 256)           24320     
_________________________________________________________________
lstm (LSTM)                  (32, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (32, None, 95)            97375     
Total params: 5,368,671
Trainable params: 5,368,671
Non-trainable params: 0
_________________________________________________________________


In [37]:
x, y = get_batch(vectorized_songs, seq_length=100, batch_size=32)
pred = model(x)
print("Input shape:      ", x.shape, " # (batch_size, sequence_length)")
print("Prediction shape: ", pred.shape, "# (batch_size, sequence_length, vocab_size)")

Input shape:       (32, 100)  # (batch_size, sequence_length)
Prediction shape:  (32, 100, 95) # (batch_size, sequence_length, vocab_size)


In [38]:
sampled_indices = tf.random.categorical(pred[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([ 8, 69, 82, 24, 59, 62, 56, 25, 46, 18, 19,  3, 55, 48, 31, 22, 64,
       84,  7, 92, 44,  8, 66, 50, 88, 15, 69, 68, 90, 24, 60, 20, 29,  9,
       20, 65,  0, 19, 81, 79, 76, 47, 31, 47, 19, 67, 92, 89, 50, 42, 93,
       86, 26, 71, 50, 37, 48, 83, 41, 51, 81, 59, 91, 10, 16,  7, 87, 71,
       56, 10, 78, 11, 63, 33, 25, 58, 88, 71, 12, 68, 54, 54, 36, 36, 10,
       20, 44, 23, 29, 38, 11, 33, 18, 52, 56, 46,  5, 66, 70, 66],
      dtype=int64)

In [39]:
print("Input: \n", repr("".join(idx2char[x[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

Input: 
 'cd dBG:|\nc>dc g2 c|e2 c g2 c|c>dc g2 c|B>cd dBG|\n|c>dc gec|edc gec|faf ege|B>cd dBG:|\n\n\nX:986\nT:Padd'

Next Char Predictions: 
 "'er7Z]W8M12!VO>5_t&|K'bQx.edz7[3<(3a\t2qolN>N2c|yQI}v9gQDOsHRqZ{)/&wgW)n*^@8Yxg+dUUCC)3K6<E*@1SWM#bfb"


In [40]:
### Defining the loss function ###

'''TODO: define the loss function to compute and return the loss between
    the true labels and predictions (logits). Set the argument from_logits=True.'''
def compute_loss(labels, logits):
  loss = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  # loss = tf.keras.losses.sparse_categorical_crossentropy('''TODO''', '''TODO''', from_logits=True) # TODO
  return loss

'''TODO: compute the loss using the true next characters from the example batch 
    and the predictions from the untrained model several cells above'''
example_batch_loss = compute_loss(y, pred)
# example_batch_loss = compute_loss('''TODO''', '''TODO''') # TODO

print("Prediction shape: ", pred.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (32, 100, 95)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5546136


In [50]:
### Hyperparameter setting and optimization ###

# Optimization parameters:
num_training_iterations = 20  # Increase this to train longer
batch_size = 4  # Experiment between 1 and 64
seq_length = 100  # Experiment between 50 and 500
learning_rate = 5e-3  # Experiment between 1e-5 and 1e-1

# Model parameters: 
vocab_size = len(vocab)
embedding_dim = 256 
rnn_units = 1024  # Experiment between 1 and 2048

# Checkpoint location: 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

In [51]:
### Define optimizer and training operation ###

'''TODO: instantiate a new model for training using the `build_model`
  function and the hyperparameters created above.'''
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)
# model = build_model('''TODO: arguments''')

'''TODO: instantiate an optimizer with its learning rate.
  Checkout the tensorflow website for a list of supported optimizers.
  https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/
  Try using the Adam optimizer to start.'''
optimizer = tf.keras.optimizers.Adam(learning_rate)
# optimizer = # TODO

@tf.function
def train_step(x, y): 
  # Use tf.GradientTape()
  with tf.GradientTape() as tape:
  
    '''TODO: feed the current input into the model and generate predictions'''
    y_hat = model(x) # TODO
    # y_hat = model('''TODO''')
  
    '''TODO: compute the loss!'''
    loss = compute_loss(y, y_hat) # TODO
    # loss = compute_loss('''TODO''', '''TODO''')

  # Now, compute the gradients 
  '''TODO: complete the function call for gradient computation. 
      Remember that we want the gradient of the loss with respect all 
      of the model parameters. 
      HINT: use `model.trainable_variables` to get a list of all model
      parameters.'''
  grads = tape.gradient(loss, model.trainable_variables) # TODO
  # grads = tape.gradient('''TODO''', '''TODO''')
  
  # Apply the gradients to the optimizer so it can update the model accordingly
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  return loss

##################
# Begin training!#
##################

# history = []
# plotter = mdl.util.PeriodicPlotter(sec=2, xlabel='Iterations', ylabel='Loss')
# if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

for iter in range(num_training_iterations):
    print(iter)

  # Grab a batch and propagate it through the network
    x_batch, y_batch = get_batch(vectorized_songs, seq_length, batch_size)
    loss = train_step(x_batch, y_batch)

  # Update the progress bar
#   history.append(loss.numpy().mean())
#   plotter.plot(history)

  # Update the model with the changed weights!
    if iter % 100 == 0:     
        model.save_weights(checkpoint_prefix)
    
# Save the trained model and the weights
model.save_weights(checkpoint_prefix)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [59]:
loss

<tf.Tensor: shape=(4, 100), dtype=float32, numpy=
array([[ 3.302258 ,  2.1601422,  2.9968367,  3.2052822,  2.2788348,
         3.2187026,  3.2187827,  2.928757 ,  2.9401193,  5.244714 ,
         2.6037123,  3.2720778,  3.60462  ,  2.9573326,  2.7897022,
         3.756322 ,  3.1845455,  2.1426778,  3.225079 ,  3.608302 ,
         2.6720371,  3.2360008,  3.3330755,  2.9354622,  2.9491124,
         3.146564 ,  2.2038898,  3.214662 ,  3.0165195,  2.396578 ,
         3.0369904,  3.0909872,  3.046271 ,  2.9559152,  5.2450614,
         2.6146529,  3.3950427,  3.232511 ,  2.9245372,  3.0002413,
         4.566617 ,  2.40705  ,  3.3784928,  2.9081833,  2.906976 ,
         3.0295649,  2.7753801,  3.2046995,  3.2406545,  2.131131 ,
         3.226939 ,  3.130402 ,  2.2493632,  3.4956126,  3.0674226,
         2.9171672,  3.7872095,  5.206357 ,  2.4688861,  3.5255089,
         4.3793173,  2.8742948,  4.594758 ,  3.1029878,  2.163796 ,
         4.3999376,  3.2202196,  2.2605808,  3.2147255,  3.219801 

In [52]:
'''TODO: Rebuild the model using a batch_size=1'''
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) # TODO
# model = build_model('''TODO''', '''TODO''', '''TODO''', batch_size=1)

# Restore the model weights for the last checkpoint after training
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (1, None, 256)            24320     
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_6 (Dense)              (1, None, 95)             97375     
Total params: 5,368,671
Trainable params: 5,368,671
Non-trainable params: 0
_________________________________________________________________


In [55]:
### Prediction of a generated song ###

def generate_text(model, start_string, generation_length=1000):
  # Evaluation step (generating ABC text using the learned RNN model)

  '''TODO: convert the start string to numbers (vectorize)'''
  input_eval = [char2idx[s] for s in start_string] # TODO
  # input_eval = ['''TODO''']
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Here batch size == 1
  model.reset_states()
#   tqdm._instances.clear()

#   for i in tqdm(range(generation_length)):
  for i in range(generation_length):
      
      '''TODO: evaluate the inputs and generate the next character predictions'''
      predictions = model(input_eval)
      # predictions = model('''TODO''')
      
      # Remove the batch dimension
      predictions = tf.squeeze(predictions, 0)
      
      '''TODO: use a multinomial distribution to sample'''
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      # predicted_id = tf.random.categorical('''TODO''', num_samples=1)[-1,0].numpy()
      
      # Pass the prediction along with the previous hidden state
      #   as the next inputs to the model
      input_eval = tf.expand_dims([predicted_id], 0)
      
      '''TODO: add the predicted character to the generated text!'''
      # Hint: consider what format the prediction is in vs. the output
      text_generated.append(idx2char[predicted_id]) # TODO 
      # text_generated.append('''TODO''')
    
  return (start_string + ''.join(text_generated))

In [56]:
generated_text = generate_text(model, start_string="X", generation_length=1000) # TODO
# generated_text = generate_text('''TODO''', start_string="X", generation_length=1000)

In [57]:
generated_text

'XXQQl322GW:~ /\'F1ly\\capdMkA\nFc~>nAt6A :re f/o|/Ag|)dN1:dbBBpr(c2/a/l|AllrA|@d(A/CBfFjdBc>:A  y|bA\n\n | " >fae).G\\ 7Df|\nsai|.mgBsc\nNr1al\nee/iBG"nd>ntc9Alp|sy,<2n| M\n||e23he8\na>\nsEEr  hFcr|\nsEG)1)fKG ln:baE sM\n )A noTl /A(Gd1edf)Dke3:|0:cB(:d25k1\' _elFjn\nA   <:dafB2 ((|c/i |\n.efuod\nkdG _i:ic  Ai)8\nuF\n/A )|"ero\n:A8A A4cM\n cie)Fn |3d:i :nsB:3\nlFiC, \n2"l/. EA<\na1\ncae ,(MG/\ndMa2r,Api:flf f|2<doA<2Es\ncGg9rn2Blz/clFcaklT\'d \nfAn:ilBfB\'>oNB:A2F/c_(e fl(E5C/1ieG2lz n:_gd)Pii/y)d(|cGdfNy Ge|:CDy j d rn  AG:(Bc bB)eltsM<bllacA 8Z)2:)cc(FZ/:F3c|ngcaToS23r")\nTc lN /fdctl.Fdlll23n1N/fkoiAAE E zcc AHc1|dF/dcseB) Ase_Z/Ge3)eG(2ng/:/Z1cdtKGadBf2.vd(Gld2:e h1mihfEF/A:NkM\nlN,dddA|l,2tC2ArebA2lA7lGGE C Cd2ein n":di)\ni)2 (/cflZ\n6h<f2 (| 2\ncfB|r1("if2)eFF\n2da:\\acTl\nabS\'"/A t2y:>e2|2 ,2Aef82mAncuu|/IF/\n0\'eesl/ eAnc\n>cel:l2c\nb@  )(f,ae|"| /h3D A cc,o\nT2 \n\n:/2\nZta etd>2nrF. a3Bc(hl\nE6D)d\'a1Me\nd\nete  FdZg|3DBB/2.MG 93sd||bcG A BE|tg/e |m\n\n\nc\n|(BagGE) Nceg|:f